In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from fedn.utils.helpers.helpers import get_helper, save_metadata, save_metrics
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
import collections

In [2]:
HELPER_MODULE = 'numpyhelper'
helper = get_helper(HELPER_MODULE)

In [3]:
def compile_model():
    peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

    peft_model = PeftModel.from_pretrained(peft_model_base, 
                                        '/Users/nash/Project/peft/peft-dialogue-summary-checkpoint-local', 
                                        torch_dtype=torch.bfloat16,
                                        is_trainable=False)
    return peft_model

In [4]:
def load_parameters(model_path):
    """ Load model parameters from file and populate model.

    param model_path: The path to load from.
    :type model_path: str
    :return: The loaded model.
    :rtype: torch.nn.Module
    """
    parameters_np = helper.load(model_path)
    model = compile_model()
    params_dict = zip(model.state_dict().keys(), parameters_np)
    state_dict = collections.OrderedDict({key: torch.tensor(x) for key, x in params_dict})
    model.load_state_dict(state_dict, strict=True)
    return model

In [5]:
in_model_path = '/Users/nash/Downloads/147c5fce-6219-4e17-a8cc-763ad9bd4028'

In [6]:
model = load_parameters(in_model_path)